In [1]:
import nltk
import numpy as np
import pandas as pd
import pre_processing_tr as pr
import networkx as nx

from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity

import fasttext
import fasttext.util

In [2]:
#Fasttext vektörlerinin yüklenmesi ve boyutunun azaltılması
ft = fasttext.load_model('cc.tr.300.bin')
ft.get_dimension()
fasttext.util.reduce_model(ft, 100)
ft.get_dimension()

100

In [3]:
#Eğer daha önceden indirmediyseniz, tek bir defaya mahsus bu kod ile indirebilirsiniz
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Omer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
#Derlemin yüklenmesi ve haber seçimi
#haber numarası adlı değişkeni dilediğiniz sayı ile değiştirip farklı haberlere erişebilirsiniz 

data = pd.read_csv('derlemler/filtrelenmis_derlem.csv.gz')
haber_numarasi = 5910
haber = data.iloc[haber_numarasi].news

In [5]:
#Eğer kendi seçtiğiniz bir haberi özetlemek isterseniz aşağıdaki satırı uncomment ediniz (#'leri kaldırın')
#ve dilediğiniz haberi noktalama işaretlerine dikkat ederek giriniz.

#haber = input("Özetlemek istediğiniz haberi giriniz:\n")

In [6]:
#Metinleri cümlelere ayırma.
cumleler = sent_tokenize(haber)
cumle_sayisi = len(cumleler)
print(f'Cümle sayısı: {cumle_sayisi}')

Cümle sayısı: 12


In [7]:
cumleler

['Onur Binay/milliyet.com.tr.',
 "Asya'da bir kameranın inanılmaz bir hikayesi ortaya çıktı.",
 'İki yıldan fazla bir süre önce okyanusta kaybolan Canon marka ve sahip olduğu fotoğraflar tekrar sahibini buldu.. Bir grup öğrenci, kuzeydeki bir plajı temizlemek için çalışıyordu.',
 '11 yaşındaki bir çocuk, tamamen taşlarla kaplı bir nesne keşfetti.',
 'Bu, su geçirmez bir muhafazası içinde kameraydı.',
 'Hatta çalışan Canon G12 dijital fotoğraf makinesiydi.',
 'Kamerayı açıp içindekilere bakan öğretmen, fotoğrafların Japonya’daki bir kadının tatildeyken farklı lokasyonlarda çektiği kareleri gördü..',
 "Son fotoğrafın 7 Eylül 2015 tarihinde Japonya'daki Ishigaki Adası'nda çekildiği anlaşılınca öğretmen, kameranın sahibini bulmak için Facebook'ta bir gönderi paylaştı.",
 'Fotoğraf makinesini sahibine ulaştırmak için yardım isteyen öğretmenin gönderisi viral oldu ve dünya çapında binlerce kişi kayıp fotoğraf makinesinin sahibini bulmak için gönderiyi paylaştı.',
 'Post dünya çapında insanla

In [8]:
temiz_cumleler = []
for i in range(cumle_sayisi):
    temiz_cumleler.append(pr.compress_whitespace(pr.remove_punc(pr.remove_numbers(str(cumleler[i])))).lower())

In [9]:
temiz_cumleler

['onur binay milliyet com tr',
 'asya da bir kameranın inanılmaz bir hikayesi ortaya çıktı',
 'i̇ki yıldan fazla bir süre önce okyanusta kaybolan canon marka ve sahip olduğu fotoğraflar tekrar sahibini buldu bir grup öğrenci kuzeydeki bir plajı temizlemek için çalışıyordu',
 'yaşındaki bir çocuk tamamen taşlarla kaplı bir nesne keşfetti',
 'bu su geçirmez bir muhafazası içinde kameraydı',
 'hatta çalışan canon g dijital fotoğraf makinesiydi',
 'kamerayı açıp içindekilere bakan öğretmen fotoğrafların japonya daki bir kadının tatildeyken farklı lokasyonlarda çektiği kareleri gördü',
 'son fotoğrafın eylül tarihinde japonya daki ishigaki adası nda çekildiği anlaşılınca öğretmen kameranın sahibini bulmak için facebook ta bir gönderi paylaştı',
 'fotoğraf makinesini sahibine ulaştırmak için yardım isteyen öğretmenin gönderisi viral oldu ve dünya çapında binlerce kişi kayıp fotoğraf makinesinin sahibini bulmak için gönderiyi paylaştı',
 'post dünya çapında insanlar tarafından on binlerce kez

In [10]:
#Cümlelerin vektöre dönüştürülmesi
cumle_vektorleri = []
for cumle in temiz_cumleler:
    if len(cumle) != 0:
        vek = np.zeros((100,))
        kelimeler = cumle.split()
        for kelime in kelimeler:
            vek = vek + (ft.get_word_vector(kelime))
        vek = vek / (len(kelimeler) + 0.1)
    else:
        vek = np.zeros((100,))
    cumle_vektorleri.append(vek)

In [11]:
#Benzerlik matrisinin oluşturulması 
sim_mat = np.zeros((cumle_sayisi, cumle_sayisi))
for i in range(cumle_sayisi):
    for j in range(cumle_sayisi):
        if i != j:
            sim_mat[i][j] = cosine_similarity(cumle_vektorleri[i].reshape(1,100), cumle_vektorleri[j].reshape(1,100))[0][0]

In [12]:
#Çizgenin (Graph) kurulması ve Page Rank algoritmasının uygulanması
nx_graph = nx.from_numpy_array(sim_mat)
skorlar = nx.pagerank(nx_graph)

In [13]:
#Cümleleri skoruna göre sıralamak (çoktan aza) 
sirali_cumleler = []
for i, j in enumerate(cumleler):
    sirali_cumleler.append((skorlar[i], j))
sirali_cumleler = sorted(sirali_cumleler, reverse=True)

In [14]:
#Özet için cümle sayısı (değiştirebilirsiniz)
#Cümle sayisindan fazla olmamalı
ozet_uzunlugu = 3

#Özet oluşturulması
ozet = ""
for i in range(ozet_uzunlugu):
    ozet = ozet + " " + sirali_cumleler[i][1]
ozet = ozet.strip()

print(ozet)

İki yıldan fazla bir süre önce okyanusta kaybolan Canon marka ve sahip olduğu fotoğraflar tekrar sahibini buldu.. Bir grup öğrenci, kuzeydeki bir plajı temizlemek için çalışıyordu. Son fotoğrafın 7 Eylül 2015 tarihinde Japonya'daki Ishigaki Adası'nda çekildiği anlaşılınca öğretmen, kameranın sahibini bulmak için Facebook'ta bir gönderi paylaştı. Asya'da bir kameranın inanılmaz bir hikayesi ortaya çıktı.
